# Query Fee Rate

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
from eth_utils import keccak
#from web3 import Web3

### Ford Subgraph

In [2]:
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

In [3]:
# function used to query each type of event from The Graph
def query_theGraphFord(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + indexer + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [5]:
# get allocationCollected event
collect_query = """query{
  allocationCollecteds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
    from
    curationFees
    rebateFees
  }
  
}"""

field_name = 'allocationCollecteds'
alloc_collected_df = query_theGraphFord(collect_query, field_name, url, False)

# save allocationCollected data
alloc_collected_df['type'] = 'allocationCollecteds'
alloc_collected_df.to_csv('allocationCollected.csv', index = False, header = True)

In [6]:
# get rewardsAssigned data
rewardsassigned_query = """query{rewardsAssigneds () {
    id
    blockNumber
    logIndex
    indexer
    amount
    epoch
}
}"""
field_name = 'rewardsAssigneds'
rewards_assigned_df = query_theGraphFord(rewardsassigned_query, field_name, url, False)

# save rewardsAssigned data
rewards_assigned_df['type'] = 'rewardsAssigneds'
rewards_assigned_df.to_csv('rewardsAssigneds.csv', index = False, header = True)

In [7]:
# get rebateClaim data
rebate_query = """query{rebateClaimeds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
    subgraphDeploymentID
    allocationID
    delegationFees
    epoch
}
}"""
field_name = 'rebateClaimeds'
rebate_claim_df = query_theGraphFord(rebate_query, field_name, url, False)

# save rebateClaim data
rebate_claim_df['type'] = 'rebateClaimeds'
rebate_claim_df.to_csv('rebateClaimeds.csv', index = False, header = True)

In [ ]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0, type:' + trans_type +'})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + str(indexer) + ', type:' + trans_type +'})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

In [ ]:
# get allocationCollected event
collect_query = """query{
  allocationCollecteds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
    from
    curationFees
    rebateFees
  }
  
}"""

field_name = 'allocationCollecteds'
trans_type = 'MintSignal'
alloc_collected_df = query_theGraph(collect_query, field_name, url, trans_type, False)

# save allocationCollected data
alloc_collected_df['type'] = 'allocationCollecteds'
alloc_collected_df.to_csv('allocationCollected.csv', index = False, header = True)

In [ ]:
# get rebateClaim data
rebate_query = """query{rebateClaimeds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
    subgraphDeploymentID
    allocationID
    delegationFees
    epoch
}
}"""
field_name = 'rebateClaimeds'
rebate_claim_df = query_theGraph(rebate_query, field_name, url, False)

# save rebateClaim data
rebate_claim_df['type'] = 'rebateClaimeds'

In [ ]:
#get mintSignal event


minted_query = '''query{
  signalTransactions() {
    id
    blockNumber
    signer{
      curator
      {
        id
      }
    }
    signal
    tokens
    type
    subgraphDeployment{
     id
    } 
  }  
}
'''

field_name = 'signalTransactions'
trans_type = 'MintSignal'

curation_mint_df = query_theGraph(minted_query, field_name, url, trans_type, False)

In [ ]:
#alloc_created_df.to_csv('allocationCreateds.csv', index = False, header = True)
curation_mint_df.head()

In [ ]:
curation_mint_df['signer'] = [i['curator']['id'] for i in curation_mint_df['signer']]
curation_mint_df['subgraphDeployment']  = [i['id'] for i in curation_mint_df['subgraphDeployment']]
curation_mint_df.head()

In [ ]:
len(curation_mint_df)

In [ ]:
#get BurnSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

created_query = '''query{
  signalTransactions() {
    id
    blockNumber
    signer{
      curator
      {
        id
      }
    }
    signal
    tokens
    type
    subgraphDeployment{
     id
    } 
  }  
}
'''

field_name = 'signalTransactions'
trans_type = 'BurnSignal'

curation_burned_df = query_theGraph(created_query, field_name, url, trans_type, False)

In [ ]:
curation_burned_df.head()

In [ ]:
curation_burned_df['signer'] = [i['curator']['id'] for i in curation_burned_df['signer']]
curation_burned_df['subgraphDeployment']  = [i['id'] for i in curation_burned_df['subgraphDeployment']]
curation_burned_df.head()

In [ ]:
# get data of existing indexer
indexer_df = pd.read_csv('../another_indexer/single_indexer/singleIndexer.csv', low_memory = False)
indexer_df.head()
#len(indexer_df)

In [ ]:
# filter events for subgraphs
subgraph_df = indexer_df[indexer_df['subgraphDeploymentID'].isna() != 1]

In [ ]:
# create list of subgraphs indexer is involved in
unique_subgraphs = subgraph_df['subgraphDeploymentID'].unique()

In [ ]:
unique_subgraphs[1]

In [ ]:
# combine mint and burn events
indexer_curation_df = curation_mint_df[curation_mint_df['subgraphDeployment'].isin(unique_subgraphs)].append(curation_burned_df[curation_burned_df['subgraphDeployment'].isin(unique_subgraphs)])

In [ ]:
indexer_curation_df.head()

In [ ]:
len(indexer_curation_df)

In [ ]:
# add curation events to existing indexer dataframe
single_indexer_df = indexer_df.append(indexer_curation_df)
single_indexer_df.sort_values(by='blockNumber')
single_indexer_df.drop(['level_0', 'index'], axis = 1, inplace = True)
single_indexer_df.head()

In [ ]:
len(single_indexer_df)

In [ ]:
# save all events for the indexer
single_indexer_df.to_csv('../another_indexer/single_indexer/singleIndexer.csv', index = False, header = True)

# GNS Exploration

In [ ]:
#get MintNSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

name_mint_query = '''query{
  nameSignalTransactions(){
    id
    blockNumber
    signer{
      curator{
        id
      }
    }
    type
    nameSignal
    versionSignal
    tokens
    subgraph{
      id
      currentVersion
      {
        version
        subgraphDeployment {
          id
        }
      }
      displayName
    }
  }
}
'''

field_name = 'nameSignalTransactions'
trans_type = 'MintNSignal'

name_mint_df = query_theGraph(name_mint_query, field_name, url, trans_type, False)

In [ ]:
name_mint_df.head()

In [ ]:
name_mint_df['curatorId'] = [i['curator']['id'] for i in name_mint_df['signer']]
name_mint_df['subgraphId'] = [i['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphDeploymentID'] = [i['currentVersion']['subgraphDeployment']['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphversion'] = [i['currentVersion']['version'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphname'] = [i['displayName'] for i in name_mint_df['subgraph']]
name_mint_df.drop(['signer','subgraph'], axis = 1, inplace = True)
name_mint_df.head()

In [ ]:
#get BurnNSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

name_burn_query = '''query{
  nameSignalTransactions(){
    id
    blockNumber
    signer{
      curator{
        id
      }
    }
    type
    nameSignal
    versionSignal
    tokens
    subgraph{
      id
      currentVersion
      {
        version
        subgraphDeployment {
          id
        }
      }
      displayName
    }
  }
}
'''

field_name = 'nameSignalTransactions'
trans_type = 'BurnNSignal'

name_burn_df = query_theGraph(name_burn_query, field_name, url, trans_type, False)

In [ ]:
name_burn_df.head()

In [ ]:
name_burn_df['curatorId'] = [i['curator']['id'] for i in name_burn_df['signer']]
name_burn_df['subgraphId'] = [i['id'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphDeploymentID'] = [i['currentVersion']['subgraphDeployment']['id'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphversion'] = [i['currentVersion']['version'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphname'] = [i['displayName'] for i in name_burn_df['subgraph']]
name_burn_df.drop(['signer','subgraph'], axis = 1, inplace = True)
name_burn_df.head()

In [ ]:
indexer_name_curation_df = name_mint_df[name_mint_df['subgraphDeploymentID'].isin(unique_subgraphs)].append(name_burn_df[name_burn_df['subgraphDeploymentID'].isin(unique_subgraphs)])

In [ ]:
#merge curation events
single_indexer_name_df = indexer_df.append(indexer_name_curation_df)
single_indexer_name_df.sort_values(by='blockNumber')
single_indexer_name_df.head()

In [ ]:
# save all events for the indexer
single_indexer_name_df.to_csv('../another_indexer/single_indexer/singleIndexername.csv', index = False, header = True)

In [ ]:
# function used to query each type of event from The Graph
def query_theGraph_subgraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:createdAt, where:{createdAt_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:createdAt, where: {createdAt_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [ ]:
#get subgraphs data
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

subgraph_query = '''query{
	subgraphs(){
    id
    owner{
      id
    }
    currentVersion{
      id
      subgraphDeployment{
        id
      }
      version
    }
    createdAt
    updatedAt
    active
    migrated
    displayName
  }
}
'''

field_name = 'subgraphs'
trans_type = 'BurnNSignal'

subgraph_df = query_theGraph_subgraph(subgraph_query, field_name, url, trans_type, False)

In [ ]:
subgraph_df.head()

In [ ]:
subgraph_df['ownerId'] = [i['id'] for i in subgraph_df['owner']]
subgraph_df['version'] = [i['version'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphDeploymentId'] = [i['subgraphDeployment']['id'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphid'] = [i['id'] for i in subgraph_df['currentVersion']]
subgraph_df.drop(['owner', 'currentVersion'], axis = 1, inplace = True)
subgraph_df.head()

In [ ]:
subgraph_df[10:20]

In [ ]:
subgraph_df['subgraph_version'] = [i.split('-')[0]+'-'+i.split('-')[1] for i in  subgraph_df['subgraphid']]
subgraph_df.head()

In [ ]:
# function used to query each type of event from The Graph
def query_theGraph_events(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [ ]:
#get grtWithdrawns event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

withdraw_query = '''query{
	grtwithdrawns(){
        id
        blockNumber
        graphAccount
        subgraphNumber
        nameCurator
        nSignalBurnt
        withdrawnGRT
  }
}
'''

field_name = 'grtwithdrawns'

withdraw_df = query_theGraph_events(withdraw_query, field_name, url, trans_type, False)

In [ ]:
withdraw_df.head()

In [ ]:
withdraw_df['subgraph_w_version'] = withdraw_df['graphAccount']+'-'+withdraw_df['subgraphNumber']
withdraw_df.head()

In [ ]:
withdraw_df['subgraphDeploymentID'] = 0

In [ ]:
for i, j in enumerate(withdraw_df['subgraph_w_version']):
    for k, l in enumerate(subgraph_df['subgraph_version']):
        if j in l:
            withdraw_df['subgraphDeploymentID'][i] = subgraph_df.iloc[k]['subgraphDeploymentId']
withdraw_df['type'] = 'grtwithdrawns'
withdraw_df.head()

In [ ]:
withdraw_df[withdraw_df['subgraphDeploymentID'].isin(unique_subgraphs)]

In [ ]:
single_indexer_name_df = single_indexer_name_df.append(withdraw_df[withdraw_df['subgraphDeploymentID'].isin(unique_subgraphs)])
single_indexer_name_df['blockNumber'] = single_indexer_name_df['blockNumber'].astype(int)
single_indexer_name_df.drop(['index'], axis = 1, inplace = True)
single_indexer_name_df.reset_index(inplace = True)
single_indexer_name_df['epoch'] = ((single_indexer_name_df['blockNumber'] - 11446768)/6646)
single_indexer_name_df['epoch'] = single_indexer_name_df['epoch'].astype(int)
single_indexer_name_df.sort_values(by=['blockNumber', 'epoch'], inplace = True)
single_indexer_name_df.head()

In [ ]:
single_indexer_name_df.columns

In [ ]:
single_indexer_name_df['type'].unique()

In [ ]:
# save all events for the indexer
single_indexer_name_df.to_csv('../another_indexer/single_indexer/singleIndexername.csv', index = False, header = True)

In [ ]:
single_indexer_name_df[['tokens', 'type']]

In [ ]:
curation_list = []
signal_list = []
epoch_list = []
for i, j in single_indexer_name_df[['tokens', 'type', 'nameSignal', 'blockNumber', 'epoch']].iterrows():
    #print(j['epoch'])
    if j['type'] == 'MintNSignal':
        curation_list.append(int(j['tokens']))
        signal_list.append(int(j['nameSignal']))
        epoch_list.append(int(j['epoch']))
    elif j['type'] == 'BurnNSignal':
        curation_list.append(-int(j['tokens']))
        signal_list.append(-int(j['nameSignal']))
        epoch_list.append(int(j['epoch']))
    else:
        curation_list.append(0)
        signal_list.append(0)
        epoch_list.append(int(j['epoch']))
curation_list = np.cumsum(curation_list)
signal_list = np.cumsum(signal_list)
plt.figure(figsize = (10, 5))
plt.plot(epoch_list , curation_list)
plt.xlabel('Epoch')
plt.ylabel('Tokens')
#plt.ylim(1e4, 1e6)
plt.title('Curation Tokens on Subgraphs Single Indexer is In')

In [ ]:
delegation_list = []
for i, j in single_indexer_name_df[['tokens', 'type', 'blockNumber']].iterrows():
    if j['type'] == 'stakeDelegateds':
        delegation_list.append(int(j['tokens']))
    elif j['type'] == 'stakeDelegatedWithdrawns':
        delegation_list.append(-int(j['tokens']))
    else:
        delegation_list.append(0)
delegation_list = np.cumsum(delegation_list)


In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(single_indexer_name_df.epoch, signal_list)
plt.xlabel('Timestep')
plt.ylabel('Tokens')
#plt.ylim(1e4, 1e6)
plt.title('Amount of Signal Curators Have for Subgraphs Single Indexer is In')

In [ ]:
token_list = ['Curated Tokens', 'Delegated Tokens']
total = {}
total['Curated Tokens'] = np.array(curation_list, dtype=float)
total['Delegated Tokens'] = np.array(delegation_list, dtype=float)
fig, ax = plt.subplots(figsize=(10, 5))
ax.stackplot(list(single_indexer_name_df.epoch), total.values(), labels = total.keys(), alpha = 0.8)
ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1.0))
ax.set_xlabel('Epoch')
ax.set_ylabel('Tokens')
ax.set_title('Delegated Tokens to Indexer vs Curated Tokens on Subgraphs Indexer is In');

Here we see that there is not much of an effect on delegation as curation events happen, as the amount of curation with respect to delegation is very small. 